# Twitter US Airlines sentiment Analysis

**Business Problem :** Create a classification model which analyze the sentiment from twitter comments

**Dataset location :**[Kaggle](https://www.kaggle.com/crowdflower/twitter-airline-sentiment)

**Description from Kaggle :**

This data originally came from Crowdflower's Data for Everyone library.

As the original source says,

A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").

The data we're providing on Kaggle is a slightly reformatted version of the original source. It includes both a CSV file and SQLite database. The code that does these transformations is available on GitHub

For example, it contains whether the sentiment of the tweets in this set was positive, neutral, or negative for six US airlines:


Lets get started,
As this is the text classification NLP problem , we are going to solve this problem with Deep learning library called as Keras

In [3]:
import tf.keras
import pandas as pd
import numpy as np
import re
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from tf.keras.preprocessing.text import Tokenizer
from tf.keras.preprocessing.sequence import pad_sequences
from tf.keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tf.keras import models
from tf.keras import layers
#from tensorflow.python import keras
import tf.keras
from tf.keras.initializers import Constant
from tf.keras.layers import LSTM
import pandas_profiling as pp

ModuleNotFoundError: No module named 'tf'

## Load the dataset

In [2]:
data = pd.read_csv('tweets.csv')

## Exploratory Data analysis 

In [3]:
pp.ProfileReport(data)

Number of variables,15
Number of observations,14640
Total Missing (%),28.2%
Total size in memory,1.7 MiB
Average record size in memory,120.0 B
Numeric,4
Categorical,11
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [4]:
df = data[['text', 'airline_sentiment']]

In [5]:
pp.ProfileReport(df)

Number of variables,2
Number of observations,14640
Total Missing (%),0.0%
Total size in memory,228.8 KiB
Average record size in memory,16.0 B
Numeric,0
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


## dropping the duplicate data

In [6]:
%time df.drop_duplicates(subset ="text",keep=False,inplace=True)
df.shape

Wall time: 98.5 ms


C:\Project\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(14240, 2)

In [7]:
df = df.reset_index(drop=True)

In [8]:
x=data['text']

In [ ]:
import string
import re
def data_clean1(data):
    text  = "".join([char for char in data if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text



from nltk.corpus import stopwords
def stopword(data):
    
    stopwordlist= stopwords.words('english')
    
    data = data.split()
    clean_data = [word for word in data if (word not in stopwordlist)]
    return " ".join(clean_data)


from textblob import Word
def lem_apply(data):
    
    data=data.split()
    new_data = [Word(word).lemmatize()  for word in data]
    return " ".join(new_data)

In [ ]:
%time df.text = df.text.apply(lambda x :data_clean1(x) )
%time df.text = df.text.apply(lambda x :stopword(x))
%time df.text = df.text.apply(lambda x :lem_apply(x))

## splitting of Dataset

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(df.text, df.airline_sentiment, test_size=0.3, random_state=142)

## Tokinizing and data filtering with keras API

In [14]:
size_words=11000
ks_tok = Tokenizer(num_words=size_words,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789',lower=True,split=" ")
ks_tok.fit_on_texts(X_train)

## convering word to sequence of number

In [15]:
X_train_seq = ks_tok.texts_to_sequences(X_train) # convert each sentence into number 
X_test_seq = ks_tok.texts_to_sequences(X_test)

# also check fit_text_tokenizer, sequences_to_matrix, text_tokenizer, texts_to_matrix, texts_to_sequences_generator, texts_to_sequences



In [16]:
max_count = X_train.apply(lambda x: len(x.split(' ')))
max_count.describe()

count    9968.000000
mean       11.689908
std         4.403139
min         2.000000
25%         8.000000
50%        12.000000
75%        15.000000
max        29.000000
Name: text, dtype: float64

## Padding with zero and making same lenght for each sequence of number

In [17]:
X_train_seq_trunc = pad_sequences(X_train_seq, maxlen=28)
X_test_seq_trunc = pad_sequences(X_test_seq, maxlen=28)

In [18]:
X_test_seq_trunc[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        460,    3,    2,  531, 3056,  278,   45,    2,   28,   32,  531,
        292,    1,  658,    8, 3056,  352])

## Label encoding of Target feature

In [19]:
le = LabelEncoder()
y_train_le = le.fit_transform(Y_train)
y_test_le = le.transform(Y_test)
y_train_oh = to_categorical(y_train_le)
y_test_oh = to_categorical(y_test_le)


## Neural Network with Keras

In [209]:
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 16 ,input_length=28 )(deep_inputs)
embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Dense(100, activation='sigmoid')(embedding)
embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_33 (Embedding)     (None, 28, 16)            176000    
_________________________________________________________________
dropout_23 (Dropout)         (None, 28, 16)            0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 448)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 3)                 1347      
Total params: 177,347
Trainable params: 177,347
Non-trainable params: 0
_________________________________________________________________


In [210]:
deep_model.fit(X_train_seq_trunc,y_train_oh,epochs=100,batch_size=512)

Epoch 1/100
9968/9968 [==============================] - 3s 297us/step - loss: 0.9964 - acc: 0.6223
Epoch 2/100
9968/9968 [==============================] - 0s 15us/step - loss: 0.9008 - acc: 0.6326
Epoch 3/100
9968/9968 [==============================] - 0s 16us/step - loss: 0.8595 - acc: 0.6327
Epoch 4/100
9968/9968 [==============================] - 0s 16us/step - loss: 0.8227 - acc: 0.6359
Epoch 5/100
9968/9968 [==============================] - 0s 15us/step - loss: 0.7816 - acc: 0.6497
Epoch 6/100
9968/9968 [==============================] - 0s 15us/step - loss: 0.7459 - acc: 0.6667
Epoch 7/100
9968/9968 [==============================] - 0s 18us/step - loss: 0.7107 - acc: 0.6850
Epoch 8/100
9968/9968 [==============================] - 0s 17us/step - loss: 0.6771 - acc: 0.7014
Epoch 9/100
9968/9968 [==============================] - 0s 15us/step - loss: 0.6458 - acc: 0.7190
Epoch 10/100
9968/9968 [==============================] - 0s 15us/step - loss: 0.6172 - acc: 0.7431
Epoch 11

In [211]:
result=deep_model.evaluate(X_train_seq_trunc,y_train_oh)
result

9968/9968 [==============================] - 1s 108us/step


[0.03749603166298741, 0.9919743178170144]

In [212]:
result=deep_model.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 0s 19us/step


[0.7948562879017677, 0.7668539325842697]

## Pretrained Glove

In [110]:
### Glove embedding
GLOVE_DIM=100

glove_file = 'glove.twitter.27B.' + str(GLOVE_DIM) + 'd.txt'
emb_dict = {}
glove = open(glove_file,encoding="utf8")
for line in glove:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    emb_dict[word] = vector
glove.close()

In [54]:
emb_matrix=np.zeros((10000,100))

In [55]:
for w,i in ks_tok.word_index.items():
    if i < 10000:
        vect = emb_dict.get(w)
        emb_matrix[i]= vect
    
    else:
        break

In [56]:
nonzero_elements = np.count_nonzero(np.count_nonzero(emb_matrix, axis=1))
result=nonzero_elements / 10000
result

0.9999

## Neural network with Glove word embedding

In [170]:
deep_inputs = layers.Input(shape=(28,),dtype='float32')
embedding= layers.Embedding(11000,100,embeddings_initializer=Constant(emb_matrix),input_length=28,trainable=True )(deep_inputs)
embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Dense(100, activation='sigmoid')(embedding)
embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3)(embedding)
deep_model_glove = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_glove.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_glove.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_24 (Embedding)     (None, 28, 100)           1100000   
_________________________________________________________________
dropout_18 (Dropout)         (None, 28, 100)           0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 2800)              0         
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 8403      
Total params: 1,108,403
Trainable params: 1,108,403
Non-trainable params: 0
_________________________________________________________________


In [171]:
deep_model_glove.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=128)

Epoch 1/50
9968/9968 [==============================] - 5s 459us/step - loss: nan - acc: 0.6320
Epoch 2/50
9968/9968 [==============================] - 2s 190us/step - loss: nan - acc: 0.63260s - loss: nan -
Epoch 3/50
9968/9968 [==============================] - 2s 157us/step - loss: nan - acc: 0.6326
Epoch 4/50
9968/9968 [==============================] - 2s 158us/step - loss: nan - acc: 0.6326
Epoch 5/50
9968/9968 [==============================] - 2s 158us/step - loss: nan - acc: 0.6326
Epoch 6/50
9968/9968 [==============================] - 2s 160us/step - loss: nan - acc: 0.6326
Epoch 7/50
9968/9968 [==============================] - 2s 162us/step - loss: nan - acc: 0.6326
Epoch 8/50
9968/9968 [==============================] - 2s 157us/step - loss: nan - acc: 0.6326
Epoch 9/50
9968/9968 [==============================] - 2s 160us/step - loss: nan - acc: 0.6326
Epoch 10/50
9968/9968 [==============================] - 2s 161us/step - loss: nan - acc: 0.6326
Epoch 11/50
9968/9968 [

## Deep learning with LSTM with Keras

In [172]:
## LSTM
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.LSTM(100,dropout=0.5, recurrent_dropout=0.5)(embedding)
embedding = layers.Dense(256, activation='sigmoid')(embedding)
embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_lstm = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_lstm.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_lstm.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_25 (Embedding)     (None, 28, 8)             88000     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               43600     
_________________________________________________________________
dense_26 (Dense)             (None, 256)               25856     
_________________________________________________________________
dropout_19 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 3)                 771       
Total params: 158,227
Trainable params: 158,227
Non-trainable params: 0
_________________________________________________________________


In [173]:
deep_model_lstm.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Epoch 1/50
9968/9968 [==============================] - 9s 895us/step - loss: 1.0426 - acc: 0.5614
Epoch 2/50
9968/9968 [==============================] - 3s 300us/step - loss: 0.9923 - acc: 0.5975
Epoch 3/50
9968/9968 [==============================] - 3s 309us/step - loss: 0.9492 - acc: 0.6212
Epoch 4/50
9968/9968 [==============================] - 3s 302us/step - loss: 0.9305 - acc: 0.6321
Epoch 5/50
9968/9968 [==============================] - 3s 307us/step - loss: 0.9229 - acc: 0.6326
Epoch 6/50
9968/9968 [==============================] - 3s 310us/step - loss: 0.9188 - acc: 0.6326
Epoch 7/50
9968/9968 [==============================] - 3s 301us/step - loss: 0.9153 - acc: 0.6326
Epoch 8/50
9968/9968 [==============================] - 3s 320us/step - loss: 0.9143 - acc: 0.6326
Epoch 9/50
9968/9968 [==============================] - 3s 303us/step - loss: 0.9136 - acc: 0.6326
Epoch 10/50
9968/9968 [==============================] - 3s 309us/step - loss: 0.9075 - acc: 0.6326
Epoch 11/

In [174]:
result=deep_model_lstm.evaluate(X_train_seq_trunc,y_train_oh)
result

9968/9968 [==============================] - 5s 463us/step


[0.21606208466555488, 0.9278691813804173]

In [175]:
result=deep_model_lstm.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 1s 283us/step


[0.7101030747095743, 0.7734082397003745]

## Deep learning with stacked LSTM

In [176]:
# LSTM multiple
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.LSTM(256,dropout=0.5, recurrent_dropout=0.5,return_sequences = True)(embedding)
embedding = layers.LSTM(128,dropout=0.5, recurrent_dropout=0.5)(embedding)
#embedding = layers.Dense(256, activation='sigmoid')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_lstm1 = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_lstm1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_lstm1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_26 (Embedding)     (None, 28, 8)             88000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 28, 256)           271360    
_________________________________________________________________
lstm_6 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dense_28 (Dense)             (None, 3)                 387       
Total params: 556,867
Trainable params: 556,867
Non-trainable params: 0
_________________________________________________________________


In [178]:
deep_model_lstm1.fit(X_train_seq_trunc,y_train_oh,epochs=10,batch_size=512)

Epoch 1/10
9968/9968 [==============================] - 16s 2ms/step - loss: 0.2965 - acc: 0.8894
Epoch 2/10
9968/9968 [==============================] - 16s 2ms/step - loss: 0.2946 - acc: 0.8904
Epoch 3/10
9968/9968 [==============================] - 15s 2ms/step - loss: 0.2852 - acc: 0.8936
Epoch 4/10
9968/9968 [==============================] - 15s 2ms/step - loss: 0.2774 - acc: 0.8975
Epoch 5/10
9968/9968 [==============================] - 16s 2ms/step - loss: 0.2681 - acc: 0.9027
Epoch 6/10
9968/9968 [==============================] - 15s 2ms/step - loss: 0.2673 - acc: 0.9011
Epoch 7/10
9968/9968 [==============================] - 16s 2ms/step - loss: 0.2585 - acc: 0.9059
Epoch 8/10
9968/9968 [==============================] - 15s 2ms/step - loss: 0.2530 - acc: 0.9066
Epoch 9/10
9968/9968 [==============================] - 16s 2ms/step - loss: 0.2506 - acc: 0.9084
Epoch 10/10
9968/9968 [==============================] - 16s 2ms/step - loss: 0.2376 - acc: 0.9096


In [179]:
result=deep_model_lstm1.evaluate(X_train_seq_trunc,y_train_oh)
result

9968/9968 [==============================] - 10s 1ms/step


[0.17049715037115312, 0.942215088282504]

In [180]:
result=deep_model_lstm1.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 3s 764us/step


[0.7094433104500788, 0.7724719101123596]

## model building with CNN

In [183]:
# CNN
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.Conv1D(256, 5, activation='relu')(embedding)
embedding = layers.GlobalMaxPooling1D()(embedding)
#embedding = layers.Dense(128, activation='relu')(embedding)
embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_lstm1 = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_lstm1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_lstm1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_28 (Embedding)     (None, 28, 8)             88000     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 24, 256)           10496     
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 256)               0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 3)                 771       
Total params: 99,267
Trainable params: 99,267
Non-trainable params: 0
_________________________________________________________________


In [184]:
deep_model_lstm1.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Epoch 1/50
9968/9968 [==============================] - 6s 589us/step - loss: 1.0065 - acc: 0.6031
Epoch 2/50
9968/9968 [==============================] - 2s 186us/step - loss: 0.9084 - acc: 0.6326
Epoch 3/50
9968/9968 [==============================] - 2s 182us/step - loss: 0.8819 - acc: 0.6326
Epoch 4/50
9968/9968 [==============================] - 2s 176us/step - loss: 0.8523 - acc: 0.6326
Epoch 5/50
9968/9968 [==============================] - 2s 184us/step - loss: 0.8099 - acc: 0.6335
Epoch 6/50
9968/9968 [==============================] - 2s 184us/step - loss: 0.7543 - acc: 0.6469
Epoch 7/50
9968/9968 [==============================] - 2s 179us/step - loss: 0.6965 - acc: 0.6865
Epoch 8/50
9968/9968 [==============================] - 2s 166us/step - loss: 0.6454 - acc: 0.7176
Epoch 9/50
9968/9968 [==============================] - 2s 184us/step - loss: 0.6028 - acc: 0.7367
Epoch 10/50
9968/9968 [==============================] - 2s 184us/step - loss: 0.5630 - acc: 0.7613
Epoch 11/

In [185]:
result=deep_model_lstm1.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 2s 420us/step


[1.2387185895934087, 0.7556179775280899]

In [ ]:
#CNN 2d
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.Conv1D(256, 5,5, activation='relu')(embedding)
embedding = layers.GlobalMaxPooling1D()(embedding)
#embedding = layers.Dense(128, activation='relu')(embedding)
embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_lstm1 = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_lstm1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_lstm1.summary()

## Model building with GRU

In [186]:
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.GRU(100,dropout=0.5, recurrent_dropout=0.5)(embedding)
#embedding = layers.Dense(256, activation='sigmoid')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_gru = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_gru.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_gru.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_29 (Embedding)     (None, 28, 8)             88000     
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               32700     
_________________________________________________________________
dense_31 (Dense)             (None, 3)                 303       
Total params: 121,003
Trainable params: 121,003
Non-trainable params: 0
_________________________________________________________________


In [187]:
deep_model_gru.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Epoch 1/50
9968/9968 [==============================] - 8s 827us/step - loss: 1.0078 - acc: 0.5979
Epoch 2/50
9968/9968 [==============================] - 2s 231us/step - loss: 0.9072 - acc: 0.6326
Epoch 3/50
9968/9968 [==============================] - 2s 228us/step - loss: 0.8685 - acc: 0.6326
Epoch 4/50
9968/9968 [==============================] - 2s 232us/step - loss: 0.8136 - acc: 0.6434
Epoch 5/50
9968/9968 [==============================] - 2s 234us/step - loss: 0.7540 - acc: 0.6643
Epoch 6/50
9968/9968 [==============================] - 2s 233us/step - loss: 0.7075 - acc: 0.6829
Epoch 7/50
9968/9968 [==============================] - 2s 231us/step - loss: 0.6717 - acc: 0.7039
Epoch 8/50
9968/9968 [==============================] - 2s 227us/step - loss: 0.6415 - acc: 0.7179
Epoch 9/50
9968/9968 [==============================] - 2s 234us/step - loss: 0.6218 - acc: 0.7305
Epoch 10/50
9968/9968 [==============================] - 2s 233us/step - loss: 0.5914 - acc: 0.7471
Epoch 11/

In [188]:
result=deep_model_gru.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 3s 664us/step


[0.951221474435892, 0.7385299625468165]

## Model building with stacked GRU

In [189]:
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.GRU(100,dropout=0.5, recurrent_dropout=0.5,return_sequences = True)(embedding)
embedding = layers.GRU(100,dropout=0.5, recurrent_dropout=0.5)(embedding)
#embedding = layers.Dense(256, activation='sigmoid')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_gru1 = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_gru1.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_gru1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_30 (Embedding)     (None, 28, 8)             88000     
_________________________________________________________________
gru_5 (GRU)                  (None, 28, 100)           32700     
_________________________________________________________________
gru_6 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_32 (Dense)             (None, 3)                 303       
Total params: 181,303
Trainable params: 181,303
Non-trainable params: 0
_________________________________________________________________


In [190]:
deep_model_gru1.fit(X_train_seq_trunc,y_train_oh,epochs=50,batch_size=512)

Epoch 1/50
9968/9968 [==============================] - 13s 1ms/step - loss: 0.9769 - acc: 0.5978
Epoch 2/50
9968/9968 [==============================] - 5s 513us/step - loss: 0.8910 - acc: 0.6326
Epoch 3/50
9968/9968 [==============================] - 5s 526us/step - loss: 0.8259 - acc: 0.6404
Epoch 4/50
9968/9968 [==============================] - 5s 515us/step - loss: 0.7480 - acc: 0.6680
Epoch 5/50
9968/9968 [==============================] - 5s 526us/step - loss: 0.6862 - acc: 0.6919
Epoch 6/50
9968/9968 [==============================] - 5s 515us/step - loss: 0.6433 - acc: 0.7082
Epoch 7/50
9968/9968 [==============================] - 5s 521us/step - loss: 0.6112 - acc: 0.7296
Epoch 8/50
9968/9968 [==============================] - 5s 528us/step - loss: 0.5825 - acc: 0.7437
Epoch 9/50
9968/9968 [==============================] - 5s 499us/step - loss: 0.5640 - acc: 0.7545
Epoch 10/50
9968/9968 [==============================] - 5s 510us/step - loss: 0.5393 - acc: 0.7699
Epoch 11/5

In [191]:
result=deep_model_gru1.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 4s 1ms/step


[1.0604827667443493, 0.7324438202247191]

## Model building with bidirectional 

In [192]:
# Biderectional LSTM
deep_inputs = layers.Input(shape=(28,))
embedding= layers.Embedding(11000 , 8 ,input_length=28 )(deep_inputs)
embedding = layers.Bidirectional(LSTM(100,dropout=0.5, recurrent_dropout=0.5))(embedding)
#embedding = layers.Dense(256, activation='sigmoid')(embedding)
#embedding = layers.Dropout(0.5)(embedding)
#embedding = layers.Flatten()(embedding)
embedding_out = layers.Dense(3, activation='softmax')(embedding)
deep_model_bdir = keras.Model(inputs=deep_inputs,outputs=embedding_out)
deep_model_bdir.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
deep_model_bdir.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_31 (Embedding)     (None, 28, 8)             88000     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               87200     
_________________________________________________________________
dense_33 (Dense)             (None, 3)                 603       
Total params: 175,803
Trainable params: 175,803
Non-trainable params: 0
_________________________________________________________________


In [196]:
deep_model_bdir.fit(X_train_seq_trunc,y_train_oh,epochs=20,batch_size=512)

Epoch 1/20
9968/9968 [==============================] - 5s 473us/step - loss: 0.1818 - acc: 0.9360
Epoch 2/20
9968/9968 [==============================] - 5s 478us/step - loss: 0.1808 - acc: 0.9362
Epoch 3/20
9968/9968 [==============================] - 5s 477us/step - loss: 0.1712 - acc: 0.9419
Epoch 4/20
9968/9968 [==============================] - 5s 501us/step - loss: 0.1748 - acc: 0.9385
Epoch 5/20
9968/9968 [==============================] - 5s 472us/step - loss: 0.1719 - acc: 0.9418
Epoch 6/20
9968/9968 [==============================] - 5s 469us/step - loss: 0.1673 - acc: 0.9403
Epoch 7/20
9968/9968 [==============================] - 5s 544us/step - loss: 0.1585 - acc: 0.9448
Epoch 8/20
9968/9968 [==============================] - 5s 542us/step - loss: 0.1616 - acc: 0.9459
Epoch 9/20
9968/9968 [==============================] - 5s 505us/step - loss: 0.1577 - acc: 0.9458
Epoch 10/20
9968/9968 [==============================] - 5s 476us/step - loss: 0.1523 - acc: 0.9468
Epoch 11/

In [197]:
result=deep_model_bdir.evaluate(X_test_seq_trunc,y_test_oh)
result

4272/4272 [==============================] - 2s 427us/step


[1.2771495857488797, 0.7521067415730337]

## Hyperparameter optimization

#### Final results with default (evaluate on test data)
1. NN  => [0.7948562879017677, 0.7668539325842697]
2. CNN => [1.2387185895934087, 0.7556179775280899]
3. CNN2d =>
4. LSTM => [0.7101030747095743, 0.7734082397003745]
5. 2 layer LSTM = > [0.7094433104500788, 0.7724719101123596]
6. GRU => [0.951221474435892, 0.7385299625468165]
7. 2 layer GRU => [1.0604827667443493, 0.7324438202247191]
8. Bidirectional(LSTM) =>[1.2771495857488797, 0.7521067415730337] 


#### Tuning model
1. NN
2. LSTM


In [215]:
## NN , LSTM  hyperparameter tunning
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
# Parameter Tunning
#param_grid = dict(num_filters=[32, 64, 128],kernel_size=[3, 5, 7],vocab_size=[vocab_size],embedding_dim=[embedding_dim],maxlen=[maxlen])


def NN_model():
    deep_inputs = layers.Input(shape=(28,))
    embedding= layers.Embedding(11000 , 16 ,input_length=28 )(deep_inputs)
    embedding = layers.Dropout(0.5)(embedding)
    #embedding = layers.Dense(100, activation='sigmoid')(embedding)
    embedding = layers.Flatten()(embedding)
    embedding_out = layers.Dense(3, activation='softmax')(embedding)
    deep_model = keras.Model(inputs=deep_inputs,outputs=embedding_out)
    deep_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    deep_model.summary()
    return deep_model

## Parameter to optimize
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)

model = KerasClassifier(build_fn=NN_model, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_seq_trunc,y_train_oh)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

C:\Project\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_34 (Embedding)     (None, 28, 16)            176000    
_________________________________________________________________
dropout_24 (Dropout)         (None, 28, 16)            0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 448)               0         
_________________________________________________________________
dense_36 (Dense)             (None, 3)                 1347      
Total params: 177,347
Trainable params: 177,347
Non-trainable params: 0
_________________________________________________________________
Best: 0.773174 using {'batch_size': 100, 'epochs': 50}
0.765650 (0.008554) with: {'batch_size': 10, 'epochs': 10}
0.762339 (0.009410) wi

In [218]:
def NN_model1(optimizer='rmsprop'):
    deep_inputs = layers.Input(shape=(28,))
    embedding= layers.Embedding(11000 , 16 ,input_length=28 )(deep_inputs)
    embedding = layers.Dropout(0.5)(embedding)
    #embedding = layers.Dense(100, activation='sigmoid')(embedding)
    embedding = layers.Flatten()(embedding)
    embedding_out = layers.Dense(3, activation='softmax')(embedding)
    deep_model = keras.Model(inputs=deep_inputs,outputs=embedding_out)
    deep_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    deep_model.summary()
    return deep_model

## Parameter to optimize
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer,)

model = KerasClassifier(build_fn=NN_model1,batch_size=100,epochs=50, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_seq_trunc,y_train_oh)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_35 (Embedding)     (None, 28, 16)            176000    
_________________________________________________________________
dropout_25 (Dropout)         (None, 28, 16)            0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 448)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 3)                 1347      
Total params: 177,347
Trainable params: 177,347
Non-trainable params: 0
_________________________________________________________________
Best: 0.778090 using {'optimizer': 'Adamax'}
0.641051 (0.004515) with: {'optimizer': 'SGD'}
0.769964 (0.007804) with: {'optimizer': 'RMS

In [220]:
def NN_model2(init_mode='uniform'):
    deep_inputs = layers.Input(shape=(28,))
    embedding= layers.Embedding(11000 , 16 ,embeddings_initializer=init_mode,input_length=28 )(deep_inputs)
    embedding = layers.Dropout(0.5)(embedding)
    #embedding = layers.Dense(100, activation='sigmoid')(embedding)
    embedding = layers.Flatten()(embedding)
    embedding_out = layers.Dense(3,kernel_initializer=init_mode, activation='softmax')(embedding)
    deep_model = keras.Model(inputs=deep_inputs,outputs=embedding_out)
    deep_model.compile(optimizer='Adamax', loss='categorical_crossentropy', metrics=['accuracy'])
    deep_model.summary()
    return deep_model

## Parameter to optimize
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode,)

model = KerasClassifier(build_fn=NN_model2,batch_size=100,epochs=50, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X_train_seq_trunc,y_train_oh)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_37 (InputLayer)        (None, 28)                0         
_________________________________________________________________
embedding_37 (Embedding)     (None, 28, 16)            176000    
_________________________________________________________________
dropout_27 (Dropout)         (None, 28, 16)            0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 448)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 3)                 1347      
Total params: 177,347
Trainable params: 177,347
Non-trainable params: 0
_________________________________________________________________
Best: 0.780899 using {'init_mode': 'normal'}
0.776986 (0.006669) with: {'init_mode': 'uniform'}
0.778792 (0.004921) with: {'init_mode': 